In [76]:
import os
import csv, subprocess
import pandas as pd

In [78]:
project = "PAS0654" #"PCON0003"
station = 2
year = 2013
isSim = 0
anaFolder = os.path.expanduser('/users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/ARA_analysis/ARA_SourceSearch/')
step1_makePairs = anaFolder + "OSC_scripts/step1-make_ped_pairs/"
outputDir = "/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/RunSummary/A%i/%i"%(station, year)


In [80]:
file = step1_makePairs+"A%i_%i_File_Ped_Pairs.txt"%(station,year)
print(file)

/users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/ARA_analysis/ARA_SourceSearch/OSC_scripts/step1-make_ped_pairs/A2_2013_File_Ped_Pairs.txt


In [114]:
with open(file, mode='r', newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    for job in reader:
        split = os.path.split(job[0])
        run = os.path.splitext(split[1])[0].strip("event")
        dataFile = job[0]
        # else:
        #     dataFile =
        submit_command = ("sbatch "
                    "--job-name=makeSummary_{0}_{7} --output=./logs/{7}_makeSummary_{0}.out --account={6} "
                    "--export=ISSIM={1},STATION={2},DATA={3},PEDESTAL={4},OUTDIR={5} runOSC.sh").format(run,isSim,station,dataFile,job[1],outputDir, project, year)
        print(submit_command)

        # print("Submitted run%s"%run)
#         exit_status = subprocess.call(submit_command, shell=True)
#         if exit_status is 1:  # Check to make sure the job submitted
#             print("Job {0} failed to submit".format(submit_command))
        break

sbatch --job-name=makeSummary_1449_2013 --output=./logs/2013_makeSummary_1449.out --account=PAS0654 --export=ISSIM=0,STATION=2,DATA=/fs/scratch/PAS0654/jorge/ARA_data/10pct/A2/2013/run1449/event1449.root,PEDESTAL=,OUTDIR=/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/RunSummary/A2/2013 runOSC.sh


In [41]:
job[0]

'/fs/scratch/PAS0654/jorge/ARA_data/10pct/A2/2013/run1449/event1449.root'

# Nested

In [109]:
cores = 6

In [127]:
count = 0
with open(file, mode='r', newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    count_core = 0
    for job in reader:
        if(count_core==0):
            f = open("/users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/ARA_analysis/ARA_SourceSearch/OSC_scripts/step2-make_run_summary/tmpScipts/A%i_%i_tmpSubmit_%i.sh"%(station, year, count), "w+")
            f.write("#!/bin/bash\n\n")
            f.write("#SBATCH --mail-type=FAIL\n")
            f.write("#SBATCH --time=01:10:00\n\n")
            f.write("eval 'source /users/PCON0003/cond0068/.bash_profile_pitzer_cvmfs'\n")
            f.write("cd /users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/\n\n")
        dataFile = job[0]
        pedestal = job[1]
        f.write("./v2_analysis_run_summary ${ISSIM} ${STATION} ${OUTDIR} %s %s &\n"%(dataFile,pedestal))
        count_core+=1

        if(count_core==cores):
            f.write("wait\n")
            f.close()
            count_core = 0
            
            split = os.path.split(job[0])
            run = os.path.splitext(split[1])[0].strip("event")
            submit_command = ("sbatch "
                        "--job-name=makeSummary_{9}_{7} --nodes=1 --ntasks-per-node={8} --output=./logs/{7}_makeSummary_{9}.out --account={6} "
                        "--export=ISSIM={1},STATION={2},OUTDIR={5},YEAR={7} tmpScipts/A{2}_{7}_tmpSubmit_{9}.sh").format(run,isSim,station,dataFile,job[1],outputDir, project, year, cores, count)
            print(submit_command)
            count+=1

if(count_core<cores):
    f.write("wait\n")
    f.close()

    split = os.path.split(job[0])
    run = os.path.splitext(split[1])[0].strip("event")
    submit_command = ("sbatch "
                "--job-name=makeSummary_{9}_{7} --nodes=1 --ntasks-per-node={8} --output=./logs/{7}_makeSummary_{9}.out --account={6} "
                "--export=ISSIM={1},STATION={2},OUTDIR={5},YEAR={7} tmpScipts/A{2}_{7}_tmpSubmit_{9}.sh").format(run,isSim,station,dataFile,job[1],outputDir, project, year, cores, count)
    print(submit_command)

        # print("Submitted run%s"%run)
#         exit_status = subprocess.call(submit_command, shell=True)
#         if exit_status is 1:  # Check to make sure the job submitted
#             print("Job {0} failed to submit".format(submit_command))
#         break

sbatch --job-name=makeSummary_0_2013 --nodes=1 --ntasks-per-node=6 --output=./logs/2013_makeSummary_0.out --account=PAS0654 --export=ISSIM=0,STATION=2,OUTDIR=/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/RunSummary/A2/2013,YEAR=2013 tmpScipts/A2_2013_tmpSubmit_0.sh
sbatch --job-name=makeSummary_1_2013 --nodes=1 --ntasks-per-node=6 --output=./logs/2013_makeSummary_1.out --account=PAS0654 --export=ISSIM=0,STATION=2,OUTDIR=/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/RunSummary/A2/2013,YEAR=2013 tmpScipts/A2_2013_tmpSubmit_1.sh
sbatch --job-name=makeSummary_2_2013 --nodes=1 --ntasks-per-node=6 --output=./logs/2013_makeSummary_2.out --account=PAS0654 --export=ISSIM=0,STATION=2,OUTDIR=/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/RunSummary/A2/2013,YEAR=2013 tmpScipts/A2_2013_tmpSubmit_2.sh
sbatch --job-name=makeSummary_3_2013 --nodes=1 --ntasks-per-node=6 --output=./logs/2013_makeSummary_3.out --account=PAS0654 --export=ISSIM=0,STATION=2,OUTDIR=/fs/project/PAS0654/ARA_DATA/A23

In [82]:
f = open("/users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/ARA_analysis/ARA_SourceSearch/OSC_scripts/step2-make_run_summary/tmpScipts/tmpSubmit_%i.sh"%count, "w+")
f.write("#!/bin/bash\n\n")
f.write("#SBATCH --mail-type=FAIL\n")
f.write("#SBATCH --time=01:10:00\n\n")
f.write("eval 'source /users/PCON0003/cond0068/.bash_profile_pitzer_cvmfs'\n")
f.write("cd /users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/")
f.write("./v2_analysis_run_summary ${ISSIM} ${STATION} ${OUTDIR} %s %s")
f.close()

In [121]:
count_core

5